In [2]:
#Se importan las librerias con las cuales se va a desarrollar el ejercicio

#Manejo de datos
import pandas as pd
import numpy as np
import os
import glob 
import openpyxl

#Gráficos
import seaborn as sns
import matplotlib.pyplot as plt

#ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE 
from xverse.transformer import WOE
from xverse.transformer import MonotonicBinning
from xverse.ensemble import VotingSelector
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers

In [3]:
# Lee los nombres de archivos del directorio de trabajo
filenames = glob.glob('Datos/*.csv')
df = pd.DataFrame()
for filename in filenames:
    data = pd.read_csv(filename)
    print(data.shape)
    df = df.append(data)
    df.shape

(5561, 1760)
(8322, 1761)


In [4]:
df.head()

,A,AA,AC,AD,AE,AF,AG,AH,AI,AK,...,tausw28,tausw29,tausw3,tausw30,tausw4,tausw5,tausw6,tausw7,tausw8,tausw9
0,28.571,16.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.979,0.0,1.895,0.994,0.781,0.0,0.0,0.0
1,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.769,0.0,2.119,0.028,1.000,0.0,0.0,0.0
2,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.844,0.0,0.977,0.096,0.138,0.0,0.0,0.0
3,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.564,0.0,0.504,0.096,0.095,0.0,0.0,0.0
4,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.813,0.0,0.980,0.848,0.065,0.0,0.0,0.0


In [5]:
# Eliminamos la columna que ennumera del primer archivo
df_dep = df.drop(columns="Unnamed: 0")

In [6]:
# Creamos una columna enumerando cada fila
df_dep = df_dep.assign(num=[0 + i for i in range(len(df_dep))])[['num'] + df_dep.columns.tolist()]
# Guardamos las variables num y sequence
df_1 = df_dep[["num", "sequence"]]
# Eliminamos la variable sequence del dataframe depurado
df_dep.pop("sequence")
df_dep.head()

,num,A,AA,AC,AD,AE,AF,AG,AH,AI,...,tausw28,tausw29,tausw3,tausw30,tausw4,tausw5,tausw6,tausw7,tausw8,tausw9
0,0,28.571,16.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.979,0.0,1.895,0.994,0.781,0.0,0.0,0.0
1,1,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.769,0.0,2.119,0.028,1.000,0.0,0.0,0.0
2,2,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.844,0.0,0.977,0.096,0.138,0.0,0.0,0.0
3,3,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.564,0.0,0.504,0.096,0.095,0.0,0.0,0.0
4,4,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.813,0.0,0.980,0.848,0.065,0.0,0.0,0.0


In [7]:
# Asignamos la variable objetivo
y = df_dep.pop("class")

In [8]:
# Normalizamos el set de datos
names = df_dep.columns
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(df_dep)
scaled_df = pd.DataFrame(scaled_df, columns=names)

In [9]:
# Particionamos el set de datos
X_train, X_test, y_train, y_test = train_test_split(scaled_df, y, test_size = .2, random_state=42, stratify = y)

In [10]:
# Se balancean los datos de entrenamiento
sm = SMOTE(random_state=42)
sm_data_X,sm_data_y=sm.fit_sample(X_train, y_train)

In [11]:
## Convirtiendo en DataFrame
bal_data_X = pd.DataFrame(sm_data_X, columns=X_train.columns)
type(bal_data_X)
bal_data_y = pd.Series(sm_data_y)
type(bal_data_y)

pandas.core.series.Series

In [26]:
# Revisamos las clases
sns.countplot(bal_data_y)

In [27]:
# Seleccionamos las variables mas importantes a partir de votos de modelos
vs = VotingSelector()
vs.fit(bal_data_X, bal_data_y)

VotingSelector(exclude_features=None, feature_names='all',
               handle_category='woe', minimum_votes=0, no_of_features=879,
               numerical_missing_values='median',
               selection_techniques=['WOE', 'RF', 'RFE', 'ETC', 'CS', 'L_ONE'])

In [106]:
delta_crit = 4
priorizadas = vs.feature_votes_[vs.feature_votes_["Votes"]>delta_crit]["Variable_Name"].values
print("Estas son las variables más importantes, según el consenso de votos de selección,", \
      delta_crit+1," donde las columnas de interes son:",\
      priorizadas)

Estas son las variables más importantes, según el consenso de votos de selección, 5  donde las columnas de interes son: ['M' '_PolarizabilityT12' 'GearyAuto_FreeEnergy5' 'Y' 'MR'
 '_PolarizabilityD2001' 'embed_0_50' 'embed_1_41' 'embed_0_29'
 'GearyAuto_FreeEnergy16' 'embed_0_40' 'QSOSW16' 'GearyAuto_FreeEnergy15'
 'MoreauBrotoAuto_Mutability10' 'embed_0_94' '_ChargeD1025' 'N'
 'GearyAuto_ResidueASA1' 'GearyAuto_FreeEnergy2'
 'MoranAuto_Hydrophobicity11' 'embed_2_25' 'QSOSW21' 'embed_2_82'
 'embed_0_49' 'embed_2_91' 'QSOgrant24' 'QSOSW24' 'embed_2_22'
 '_PolarizabilityD3050' '_NormalizedVDWVD2050' 'GearyAuto_ResidueASA2'
 'embed_2_42' '_SolventAccessibilityD1050' 'embed_2_94'
 '_HydrophobicityC1' '_NormalizedVDWVD1100' 'C' 'embed_1_49' 'QSOSW3'
 'QSOgrant3' 'QSOgrant19' 'QSOSW19' '_HydrophobicityD3025'
 '_PolarizabilityC1' 'R' '_PolarizabilityD1100' 'embed_2_65'
 '_PolarityD2025' 'embed_2_41' 'embed_0_42' '_SecondaryStrD2025'
 'embed_2_57' 'embed_1_57' 'MK' 'embed_2_24' 'tausw4' 'taugr

## Entrenar con las caracteristicas seleccionadas

In [107]:
priorizadas.shape

(513,)

In [29]:
val_data = pd.read_csv('Datos/val/DatosValidacion1.csv')
# Creamos una columna enumerando cada fila
val_data = val_data.assign(num=[1500 + i for i in range(len(val_data))])[['num'] + val_data.columns.tolist()]
val_data.pop("sequence")
y2 = val_data.pop("class")
val_data.head()

,num,Unnamed: 0,Unnamed: 0.1,length,molecular_weight,charge,charge_density,isoelectric_point,gravy,instability_index,...,embed_2_90,embed_2_91,embed_2_92,embed_2_93,embed_2_94,embed_2_95,embed_2_96,embed_2_97,embed_2_98,embed_2_99
0,1500,0,0,33,3897.77,8.691,0.002230,11.404358,-0.112121,84.766667,...,-0.928374,2.771416,-0.851930,-0.459909,0.909622,1.402783,-3.848056,-0.528822,-0.740751,-0.257650
1,1501,1,1,34,4003.64,7.590,0.001896,10.196106,-0.638235,76.035294,...,-1.030314,2.356963,-0.590644,-0.433246,0.362768,1.204798,-3.838024,-0.859893,-1.086264,0.052278
2,1502,2,2,35,4121.92,6.689,0.001623,9.611023,-0.551429,16.851429,...,-0.846578,2.762231,-1.182640,-0.908285,0.802487,1.546229,-4.543212,-0.786475,-0.477468,-0.178319
3,1503,3,3,31,3838.56,6.589,0.001717,9.802917,-0.200000,53.977419,...,-0.960986,1.742604,-0.981736,-0.098695,0.614634,0.774174,-2.818301,-0.395545,-0.738405,0.369630
4,1504,4,4,33,3715.45,3.591,0.000967,8.981384,0.660606,62.042424,...,-1.034936,1.734715,-0.830263,-0.386610,0.836167,0.859871,-3.665342,-0.922017,-0.364447,-0.055827


In [30]:
# Normalizamos el set de datos de validación
names = val_data.columns
scaler = preprocessing.StandardScaler()
scaled_val_data = scaler.fit_transform(val_data)
scaled_val_data  = pd.DataFrame(scaled_val_data, columns=names)
scaled_val_data.head()

,num,Unnamed: 0,Unnamed: 0.1,length,molecular_weight,charge,charge_density,isoelectric_point,gravy,instability_index,...,embed_2_90,embed_2_91,embed_2_92,embed_2_93,embed_2_94,embed_2_95,embed_2_96,embed_2_97,embed_2_98,embed_2_99
0,-1.730984,-1.704958,-1.710290,1.200441,1.488957,2.993163,2.041380,1.680420,0.057637,1.795329,...,-0.709365,1.945619,-1.703364,0.675871,1.054814,3.373537,-2.357323,0.456880,-0.985170,-1.610517
1,-1.728850,-1.703898,-1.709234,1.346667,1.627166,2.600490,1.720552,1.201197,-0.688985,1.474132,...,-0.996886,1.241833,-0.840013,0.748034,-0.608378,2.709067,-2.344236,-0.497817,-2.046474,-0.469698
2,-1.726715,-1.702838,-1.708178,1.492893,1.781575,2.279148,1.458301,0.969138,-0.565795,-0.703036,...,-0.478661,1.930022,-2.796106,-0.537628,0.728976,3.854966,-3.264137,-0.286104,-0.176447,-1.318507
3,-1.724581,-1.701777,-1.707123,0.907989,1.411661,2.243483,1.548356,1.045248,-0.067074,0.662700,...,-0.801345,0.198585,-2.132271,1.653474,0.157642,1.263816,-1.014032,0.841203,-0.977964,0.698452
4,-1.722446,-1.700717,-1.706067,1.200441,1.250947,1.174245,0.827828,0.719407,1.154235,0.959383,...,-1.009921,0.185188,-1.631771,0.874250,0.831410,1.551430,-2.118977,-0.676961,0.170717,-0.867622


In [108]:
data_val_x = scaled_val_data[priorizadas]
data_val_y = y2

In [109]:
# Random Forest Validación
# Entrenamos un modelo con Random Forest 
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(bal_data_X[priorizadas], bal_data_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [110]:
# Hacemos predicciones con la data de prueba
predict = rf.predict(X_test[priorizadas])
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predict)

array([[1099,   13],
       [   8, 1657]], dtype=int64)

In [111]:
# Revisamos otras metricas del modelo
from sklearn.metrics import classification_report
print(classification_report(y_test, predict))

precision    recall  f1-score   support

           0       0.99      0.99      0.99      1112
           1       0.99      1.00      0.99      1665

    accuracy                           0.99      2777
   macro avg       0.99      0.99      0.99      2777
weighted avg       0.99      0.99      0.99      2777



In [52]:
param_grid = { 
    'n_estimators': [10, 20, 30, 50, 100, 200, 500, 1000, 2000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [1,2,3,4,5,6,7,8,9,10],
    'criterion' :['gini', 'entropy'],
    'min_samples_leaf': [3, 4, 5, 7, 10, 15, 20, 30, 50],  
    #'params': [{'kernel' : 'rbf', 'gamma' : 0.1}, ...],  
    'min_samples_split': [8, 10, 12, 16, 20, 30, 50, 100, 200]
}

In [112]:
from sklearn.model_selection import RandomizedSearchCV
rf_rdm= RandomizedSearchCV(estimator=rf, param_distributions=param_grid, cv= 3, n_iter=100, n_jobs=-1)
rf_rdm.fit(bal_data_X[priorizadas], bal_data_y)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [114]:
rf_rdm.best_params_

{'n_estimators': 1000,
 'min_samples_split': 12,
 'min_samples_leaf': 5,
 'max_features': 'auto',
 'max_depth': 9,
 'criterion': 'entropy'}

In [115]:
rf1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 1000, max_depth=9, criterion='entropy', min_samples_split = 12,min_samples_leaf = 5 )

In [116]:
rf1.fit(bal_data_X[priorizadas], bal_data_y)
predict1 = rf1.predict(data_val_x[priorizadas])
confusion_matrix(data_val_y, predict1)

array([[ 345, 1009],
       [  24,  245]], dtype=int64)

In [117]:
print(classification_report(data_val_y, predict1))

precision    recall  f1-score   support

           0       0.93      0.25      0.40      1354
           1       0.20      0.91      0.32       269

    accuracy                           0.36      1623
   macro avg       0.57      0.58      0.36      1623
weighted avg       0.81      0.36      0.39      1623



In [121]:
predict1 = rf1.predict(data_val_x[0:500])
print(confusion_matrix(data_val_y[0:500], predict1))
print(classification_report(data_val_y[0:500], predict1))

[[186  45]
 [ 24 245]]
              precision    recall  f1-score   support

           0       0.89      0.81      0.84       231
           1       0.84      0.91      0.88       269

    accuracy                           0.86       500
   macro avg       0.87      0.86      0.86       500
weighted avg       0.86      0.86      0.86       500



In [128]:
predict1 = rf1.predict(data_val_x[501:1000])
print(confusion_matrix(data_val_y[501:1000], predict1))
print(classification_report(data_val_y[501:1000], predict1))

[[127 372]
 [  0   0]]
              precision    recall  f1-score   support

           0       1.00      0.25      0.41       499
           1       0.00      0.00      0.00         0

    accuracy                           0.25       499
   macro avg       0.50      0.13      0.20       499
weighted avg       1.00      0.25      0.41       499



In [123]:
predict1 = rf1.predict(data_val_x[1001:1623])
print(confusion_matrix(data_val_y[1001:1623], predict1))
print(classification_report(data_val_y[1001:1623], predict1))

[[ 31 591]
 [  0   0]]
              precision    recall  f1-score   support

           0       1.00      0.05      0.09       622
           1       0.00      0.00      0.00         0

    accuracy                           0.05       622
   macro avg       0.50      0.02      0.05       622
weighted avg       1.00      0.05      0.09       622



In [124]:
from sklearn.metrics import accuracy_score
ini = -1
fin = 100
for i in range(17):
    ini = ini + 1
    print("Iteración:", i, ",", ini, "-", fin )
    predict1 = rf1.predict(data_val_x[ini:fin])
    print("Accuracy:", accuracy_score(data_val_y[ini:fin],predict1) )
    print(classification_report(data_val_y[ini:fin], predict1))
    ini += 99
    fin += 100 

Iteración: 0 , 0 - 100
Accuracy: 0.94
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.94      0.97       100

    accuracy                           0.94       100
   macro avg       0.50      0.47      0.48       100
weighted avg       1.00      0.94      0.97       100

Iteración: 1 , 100 - 200
Accuracy: 0.93
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.93      0.96       100

    accuracy                           0.93       100
   macro avg       0.50      0.47      0.48       100
weighted avg       1.00      0.93      0.96       100

Iteración: 2 , 200 - 300
Accuracy: 0.85
              precision    recall  f1-score   support

           0       0.71      0.87      0.78        31
           1       0.94      0.84      0.89        69

    accuracy                           0.85       100
   macro a